#### INFO


***SETUP***

1. *Runtime ->  Change Runtime Type -> Python 3 + GPU*
2. *Fill in `GDRIVE_DIR` in [code #5] (name of the folder in gdrive that contains your code and data)*

***RUNNING THE SCRIPTS***
1. Mount the drive before the first run. Also, sometimes so you may need to re-mount the drive ([code #6])
2. Tensorboard - should be enough to run the script once; however restart should work OK ([code #7])
3. Training - when re-running the training, you **do not** have to run any of the preceding scripts

***ABOUT***

- *[code #1] Install dependencies for google authentification*
- *[code #2] Allow collaboratory to read from & write into your google drive"*
- *[code #3] Download tensorboard accesories*
- *[code #4] Setup paths for tensorboard and your script.*
- *[code #5] Declares the function that will be used to run your tensorflow script.*
- *[code #6] Mounts the google drive to current FS as ./drive.*
- *[code #7] Fires up tensorboard.*
- *[code #8] VM reset script - Kills all processes. Sometimes helps, once the GPU tensorflow stops working, drive fails etc. **! RESETS AUTH !** *
- *[code #9] Sample call to run function that actually runs your script.*


***FINAL NOTES***

Most of this script was copy-pasted from https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d

The Tensorflow part is taken from https://github.com/mixuala/colab_utils.git + Special thanks to Štěpán Hojdar.

Feel free to improve this script - just send me suggestions / bug reports / improved scripts and I will upload it here.

Created by Vladislav Vancák & Petr Houška.

#### SETUP

In [1]:
# DRIVE & AUTH DEPENDENCIES
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 16712 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting up python-apt (1.4.0~beta3build2) ...
Setting up python3-dbus (1.2.4-1build3) ...
Setting up python-software-properties (0.96.24.17) ...
Setting up software-properties-common (0.96.24.17) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Processing

In [2]:
# TENSORBOARD DEPENDENCIES
!git clone https://github.com/mixuala/colab_utils.git

Cloning into 'colab_utils'...
remote: Counting objects: 216, done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 216 (delta 15), reused 40 (delta 12), pack-reused 171
Receiving objects: 100% (216/216), 60.19 KiB | 4.01 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [3]:
# GOOGLE AUTH
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


#### CONFIG

In [0]:
# SCRIPT PROPERTIES
import os

GDRIVE_DIR="DL_fashion_masks"

ROOT = %pwd
SCRIPT_PATH = os.path.join(ROOT, 'drive', GDRIVE_DIR)
LOG_DIR = os.path.join(SCRIPT_PATH, 'logs')

In [0]:
# RUNNER FUNCTION
import tensorflow as tf
def run_script(name, parameters):
  SCRIPT_FILE= name
  SCRIPT_ARGS= parameters

  os.chdir(SCRIPT_PATH)

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    print('[WARN]: GPU device not found. Running in CPU mode')
  else:
    print('[INFO]: Found GPU at: {}'.format(device_name))  
  print('<<= <=> =>>')
  
  %run $SCRIPT_FILE $SCRIPT_ARGS

#### GOOGLE DRIVE MOUNT & TENSORBOARD

In [0]:
# MOUNT DRIVE / CLEAR CACHE
import os
os.chdir(ROOT)
  
if os.path.exists('drive'):
  !google-drive-ocamlfuse -cc

else:
  !mkdir -p drive
  !google-drive-ocamlfuse drive

In [7]:
# TENSORBOARD
import os
import sys
import colab_utils.tboard
  
os.chdir(ROOT)
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR)

calling wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip ...
calling unzip ngrok-stable-linux-amd64.zip ...
ngrok installed. path=/content/ngrok
status: tensorboard=False, ngrok=False
tensorboard url= https://e238a654.ngrok.io


'https://e238a654.ngrok.io'

#### UTILS

In [0]:
# RESETS VM
!kill -9 -1 

#### RUN

In [13]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 20\
    --batch_size 256\
    --logname default\
    --learning_rate 0.002 --learning_rate_final 0.0001\
    --cnn_c C-64-3-1-same,C-64-3-1-same,M-3-2,C-128-3-1-same,C-128-3-1-same,M-3-2\
    --cnn_m C-128-3-1-same,C-128-3-1-same,F,R-1024,D\
    --cnn_l F,R-512,D\
    "
)

[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.8472|iou:0.7377
1|acc:0.8736|iou:0.7786
2|acc:0.8854|iou:0.7980
3|acc:0.8888|iou:0.8052
4|acc:0.8924|iou:0.8130
5|acc:0.8968|iou:0.8195
6|acc:0.8942|iou:0.8202
7|acc:0.9008|iou:0.8283
8|acc:0.8998|iou:0.8278
9|acc:0.9026|iou:0.8324
10|acc:0.9030|iou:0.8341
11|acc:0.9062|iou:0.8377
12|acc:0.9024|iou:0.8350
13|acc:0.9076|iou:0.8406
14|acc:0.9048|iou:0.8381
15|acc:0.9064|iou:0.8409
16|acc:0.9078|iou:0.8425
17|acc:0.9052|iou:0.8401
18|acc:0.9084|iou:0.8437
19|acc:0.9068|iou:0.8422


In [14]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 50\
    --batch_size 256\
    --logname default_CB_fatter\
    --learning_rate 0.002 --learning_rate_final 0.0001\
    --cnn_c CB-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2\
    --cnn_m CB-256-3-1-same,CB-256-3-1-same,F,R-2048,D\
    --cnn_l F,R-512,D\
    "
)


[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.6512|iou:0.5194
1|acc:0.8062|iou:0.6740
2|acc:0.8570|iou:0.7285
3|acc:0.8676|iou:0.7396
4|acc:0.8540|iou:0.7317
5|acc:0.8718|iou:0.7492
6|acc:0.8852|iou:0.7635
7|acc:0.9058|iou:0.7854
8|acc:0.8918|iou:0.7710
9|acc:0.8978|iou:0.7787
10|acc:0.8802|iou:0.7620
11|acc:0.8890|iou:0.7746
12|acc:0.9130|iou:0.7953
13|acc:0.8988|iou:0.7846
14|acc:0.9026|iou:0.7903
15|acc:0.9122|iou:0.7993
16|acc:0.9212|iou:0.8075
17|acc:0.9174|iou:0.8057
18|acc:0.9060|iou:0.7949
19|acc:0.9180|iou:0.8075
20|acc:0.9184|iou:0.8086
21|acc:0.9172|iou:0.8077
22|acc:0.9148|iou:0.8047
23|acc:0.9268|iou:0.8166
24|acc:0.9218|iou:0.8123
25|acc:0.9248|iou:0.8164
26|acc:0.9248|iou:0.8163
27|acc:0.9232|iou:0.8157
28|acc:0.9260|iou:0.8183
29|acc:0.9256|iou:0.8174


In [15]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 40\
    --batch_size 256\
    --logname default_C_fatter\
    --learning_rate 0.002 --learning_rate_final 0.0001\
    --cnn_c C-128-3-1-same,C-128-3-1-same,M-3-2,C-256-3-1-same,C-256-3-1-same,M-3-2\
    --cnn_m C-256-3-1-same,C-256-3-1-same,F,R-2048,D\
    --cnn_l F,R-512,D\
    "
)


[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.2436|iou:0.1629
1|acc:0.7528|iou:0.6557
2|acc:0.7606|iou:0.6845
3|acc:0.8292|iou:0.7518
4|acc:0.8326|iou:0.7574
5|acc:0.8514|iou:0.7821
6|acc:0.8660|iou:0.7991
7|acc:0.8614|iou:0.7959
8|acc:0.8618|iou:0.8004
9|acc:0.8680|iou:0.8068
10|acc:0.8798|iou:0.8199
11|acc:0.8766|iou:0.8172
12|acc:0.8812|iou:0.8236
13|acc:0.8846|iou:0.8281
14|acc:0.8810|iou:0.8255
15|acc:0.8838|iou:0.8296
16|acc:0.8876|iou:0.8336
17|acc:0.8876|iou:0.8336
18|acc:0.8854|iou:0.8331
19|acc:0.8936|iou:0.8411
20|acc:0.8926|iou:0.8404
21|acc:0.8940|iou:0.8426
22|acc:0.8912|iou:0.8399
23|acc:0.8958|iou:0.8448
24|acc:0.8968|iou:0.8460
25|acc:0.8950|iou:0.8447
26|acc:0.8952|iou:0.8452
27|acc:0.8964|iou:0.8464
28|acc:0.8988|iou:0.8488
29|acc:0.8978|iou:0.8483


In [0]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 30\
    --batch_size 256\
    --logname default_CB_longer_common\
    --learning_rate 0.002 --learning_rate_final 0.0001\
    --cnn_c CB-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,C-512-3-1-same,CB-512-3-1-same,M-3-2\
    --cnn_m CB-256-3-1-same,CB-256-3-1-same,F,R-2048,D\
    --cnn_l F,R-512,D\
    "
)


[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.5196|iou:0.3590
1|acc:0.6908|iou:0.5459
2|acc:0.8314|iou:0.7184
3|acc:0.8164|iou:0.7050
4|acc:0.8822|iou:0.7770
5|acc:0.8634|iou:0.7552
6|acc:0.8904|iou:0.7907
7|acc:0.8700|iou:0.7807
8|acc:0.8782|iou:0.7763
9|acc:0.9178|iou:0.8284
10|acc:0.8784|iou:0.7927
11|acc:0.9100|iou:0.8255
12|acc:0.9168|iou:0.8322
13|acc:0.9120|iou:0.8310
14|acc:0.9106|iou:0.8311
15|acc:0.9222|iou:0.8447
16|acc:0.9246|iou:0.8496
17|acc:0.9296|iou:0.8489
18|acc:0.9284|iou:0.8542
19|acc:0.9232|iou:0.8509
20|acc:0.9266|iou:0.8548
21|acc:0.9236|iou:0.8522
22|acc:0.9260|iou:0.8558


In [10]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 40\
    --batch_size 256\
    --logname default_CB_longer_common_longer_all\
    --learning_rate 0.002 --learning_rate_final 0.0001\
    --cnn_c CB-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,C-512-3-1-same,CB-512-3-1-same,M-3-2\
    --cnn_m CB-512-3-1-same,CB-512-3-1-same,CB-512-3-1-same,F,R-2048,D\
    --cnn_l F,R-512,D,R-64,D\
    "
)


[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.0980|iou:0.0616
1|acc:0.3848|iou:0.3213
2|acc:0.7258|iou:0.6051
3|acc:0.8188|iou:0.6890
4|acc:0.8536|iou:0.7287
5|acc:0.8038|iou:0.6788
6|acc:0.8766|iou:0.7624
7|acc:0.9020|iou:0.7837
8|acc:0.8886|iou:0.7841
9|acc:0.8880|iou:0.7843
10|acc:0.8456|iou:0.7488
11|acc:0.8864|iou:0.7866
12|acc:0.9082|iou:0.8135
13|acc:0.9134|iou:0.8182
14|acc:0.9166|iou:0.8236
15|acc:0.9066|iou:0.8183
16|acc:0.9222|iou:0.8347
17|acc:0.9104|iou:0.8221
18|acc:0.9222|iou:0.8369
19|acc:0.9180|iou:0.8278
20|acc:0.8922|iou:0.8069
21|acc:0.9184|iou:0.8384
22|acc:0.9208|iou:0.8429
23|acc:0.9206|iou:0.8431
24|acc:0.9256|iou:0.8493
25|acc:0.9256|iou:0.8482
26|acc:0.9184|iou:0.8439
27|acc:0.9258|iou:0.8521
28|acc:0.9208|iou:0.8479
29|acc:0.9288|iou:0.8577
30|acc:0.9260|iou:0.8536
31|acc:0.9280|iou:0.8577
32|acc:0.9292|iou:0.8593
33|acc:0.9264|iou:0.8558
34|acc:0.9312|iou:0.8627
35|acc:0.9308|iou:0.8625
36|acc:0.9302|iou:0.8621
37|acc:0.9298|iou:0.8618
38|acc:0.928

In [14]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 70\
    --batch_size 256\
    --logname default_CB_longer_common_longer_all_more_epochs\
    --learning_rate 0.002 --learning_rate_final 0.0001\
    --cnn_c CB-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,C-512-3-1-same,CB-512-3-1-same,M-3-2\
    --cnn_m CB-512-3-1-same,CB-512-3-1-same,CB-512-3-1-same,F,R-2048,D\
    --cnn_l F,R-512,D,R-64,D\
    "
)


[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.0980|iou:0.0616
1|acc:0.2996|iou:0.2485
2|acc:0.6788|iou:0.5246
3|acc:0.8234|iou:0.7005
4|acc:0.8356|iou:0.7222
5|acc:0.8540|iou:0.7431
6|acc:0.8856|iou:0.7725
7|acc:0.8862|iou:0.7775
8|acc:0.8818|iou:0.7727
9|acc:0.8794|iou:0.7810
10|acc:0.8680|iou:0.7631
11|acc:0.8932|iou:0.7971
12|acc:0.8638|iou:0.7687
13|acc:0.9046|iou:0.8080
14|acc:0.9090|iou:0.8197
15|acc:0.9024|iou:0.8107
16|acc:0.8938|iou:0.8059
17|acc:0.9118|iou:0.8283
18|acc:0.9172|iou:0.8354
19|acc:0.8912|iou:0.8051
20|acc:0.9170|iou:0.8326
21|acc:0.9184|iou:0.8362
22|acc:0.9248|iou:0.8492
23|acc:0.9194|iou:0.8419
24|acc:0.9116|iou:0.8317
25|acc:0.9234|iou:0.8489
26|acc:0.9188|iou:0.8431
27|acc:0.9240|iou:0.8527
28|acc:0.9262|iou:0.8545
29|acc:0.9208|iou:0.8489
30|acc:0.9196|iou:0.8490
31|acc:0.9234|iou:0.8522
32|acc:0.9210|iou:0.8507
33|acc:0.9276|iou:0.8590
34|acc:0.9250|iou:0.8574
35|acc:0.9236|iou:0.8551
36|acc:0.9254|iou:0.8585
37|acc:0.9240|iou:0.8580
38|acc:0.927

In [0]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 40\
    --batch_size 256\
    --logname default_CB_longer_common_longer_all_longest_masks\
    --learning_rate 0.002 --learning_rate_final 0.0001\
    --cnn_c CB-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,C-512-3-1-same,CB-512-3-1-same,M-3-2\
    --cnn_m CB-512-3-1-same,CB-512-3-1-same,CB-256-3-1-same,CB-128-3-1-same,F,R-4096,D\
    --cnn_l F,R-512,D,R-64,D\
    "
)


[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.0980|iou:0.0613
1|acc:0.5074|iou:0.4025
2|acc:0.7544|iou:0.6235
3|acc:0.8390|iou:0.7093
4|acc:0.8668|iou:0.7460
5|acc:0.8914|iou:0.7755
6|acc:0.8688|iou:0.7623
7|acc:0.9002|iou:0.7943
8|acc:0.8934|iou:0.7837
9|acc:0.9082|iou:0.8021
10|acc:0.8678|iou:0.7607
11|acc:0.8882|iou:0.7798
12|acc:0.9168|iou:0.8176
13|acc:0.9032|iou:0.8021
14|acc:0.9066|iou:0.8145
15|acc:0.9202|iou:0.8267
16|acc:0.9134|iou:0.8255
17|acc:0.9124|iou:0.8191
18|acc:0.9128|iou:0.8221


In [13]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 45\
    --batch_size 256\
    --logname default_CB_longer_common_even_longer\
    --learning_rate 0.002 --learning_rate_final 0.0001\
    --cnn_c CB-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,C-512-3-1-same,CB-512-3-1-same,M-3-2,CB-512-3-1-same\
    --cnn_m CB-512-3-1-same,CB-512-3-1-same,F,R-2048,D\
    --cnn_l F,R-512,D\
    "
)


[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.4608|iou:0.3452
1|acc:0.7614|iou:0.6148
2|acc:0.8704|iou:0.7323
3|acc:0.8902|iou:0.7638
4|acc:0.8960|iou:0.7751
5|acc:0.8646|iou:0.7451
6|acc:0.8784|iou:0.7647
7|acc:0.8954|iou:0.7800
8|acc:0.9048|iou:0.7914
9|acc:0.8968|iou:0.7866
10|acc:0.8486|iou:0.7383
11|acc:0.8906|iou:0.7824
12|acc:0.9180|iou:0.8126
13|acc:0.9072|iou:0.8056
14|acc:0.8824|iou:0.7889
15|acc:0.9104|iou:0.8104
16|acc:0.9202|iou:0.8245
17|acc:0.9194|iou:0.8299
18|acc:0.9190|iou:0.8308
19|acc:0.9142|iou:0.8299
20|acc:0.9222|iou:0.8370
21|acc:0.9184|iou:0.8348
22|acc:0.9270|iou:0.8457
23|acc:0.9226|iou:0.8381
24|acc:0.9192|iou:0.8381
25|acc:0.9236|iou:0.8459
26|acc:0.9268|iou:0.8488
27|acc:0.9250|iou:0.8484
28|acc:0.9268|iou:0.8510
29|acc:0.9186|iou:0.8417
30|acc:0.9244|iou:0.8507
31|acc:0.9284|iou:0.8550
32|acc:0.9250|iou:0.8522
33|acc:0.9274|iou:0.8556
34|acc:0.9290|iou:0.8579
35|acc:0.9280|iou:0.8580
36|acc:0.9270|iou:0.8570
37|acc:0.9288|iou:0.8595
38|acc:0.924

In [0]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 60\
    --batch_size 256\
    --logname default_CB_longer_common_more_epochs\
    --learning_rate 0.002 --learning_rate_final 0.0001\
    --cnn_c CB-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,C-512-3-1-same,CB-512-3-1-same,M-3-2\
    --cnn_m CB-256-3-1-same,CB-256-3-1-same,F,R-2048,D\
    --cnn_l F,R-512,D\
    "
)


[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.5924|iou:0.4472
1|acc:0.7210|iou:0.5889
2|acc:0.7978|iou:0.6774
3|acc:0.8400|iou:0.7335
4|acc:0.8312|iou:0.7268
5|acc:0.8870|iou:0.7808
6|acc:0.8598|iou:0.7633
7|acc:0.8718|iou:0.7750
8|acc:0.8398|iou:0.7464
9|acc:0.9006|iou:0.8037
10|acc:0.8998|iou:0.8100
11|acc:0.9162|iou:0.8288
12|acc:0.8856|iou:0.7957
13|acc:0.9062|iou:0.8210
14|acc:0.9138|iou:0.8334
15|acc:0.8966|iou:0.8139
16|acc:0.8974|iou:0.8163
17|acc:0.9176|iou:0.8378
18|acc:0.9126|iou:0.8316
19|acc:0.9108|iou:0.8349
20|acc:0.9256|iou:0.8520
21|acc:0.9158|iou:0.8448
22|acc:0.9268|iou:0.8558
23|acc:0.9204|iou:0.8501
24|acc:0.9178|iou:0.8456
25|acc:0.9226|iou:0.8549
26|acc:0.9084|iou:0.8344
27|acc:0.9236|iou:0.8575
28|acc:0.9294|iou:0.8643
29|acc:0.9298|iou:0.8646
30|acc:0.9260|iou:0.8620
31|acc:0.9264|iou:0.8641
32|acc:0.9270|iou:0.8633


In [8]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 80\
    --batch_size 256\
    --logname default_CB_longer_common_most_epochs\
    --learning_rate 0.0015 --learning_rate_final 0.0001\
    --cnn_c CB-128-3-1-same,CB-128-3-1-same,M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,C-512-3-1-same,CB-512-3-1-same,M-3-2\
    --cnn_m CB-256-3-1-same,CB-256-3-1-same,F,R-2048,D\
    --cnn_l F,R-512,D\
    "
)


[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.5974|iou:0.4635
1|acc:0.8208|iou:0.6894
2|acc:0.8246|iou:0.7172
3|acc:0.8712|iou:0.7639
4|acc:0.8384|iou:0.7376
5|acc:0.8722|iou:0.7698
6|acc:0.8550|iou:0.7588
7|acc:0.8306|iou:0.7390
8|acc:0.8782|iou:0.7882
9|acc:0.9142|iou:0.8232
10|acc:0.8996|iou:0.8166
11|acc:0.9126|iou:0.8270
12|acc:0.9022|iou:0.8135
13|acc:0.9014|iou:0.8138
14|acc:0.9030|iou:0.8143
15|acc:0.8992|iou:0.8202
16|acc:0.8998|iou:0.8229
17|acc:0.9100|iou:0.8246
18|acc:0.9176|iou:0.8437
19|acc:0.8998|iou:0.8265
20|acc:0.9168|iou:0.8436
21|acc:0.9192|iou:0.8497
22|acc:0.9120|iou:0.8420
23|acc:0.9192|iou:0.8503
24|acc:0.9238|iou:0.8558
25|acc:0.9158|iou:0.8491
26|acc:0.9086|iou:0.8414
27|acc:0.9198|iou:0.8561
28|acc:0.9218|iou:0.8577
29|acc:0.9294|iou:0.8675
30|acc:0.9176|iou:0.8505
31|acc:0.9236|iou:0.8633
32|acc:0.9268|iou:0.8655
33|acc:0.8952|iou:0.8325
34|acc:0.9254|iou:0.8653
35|acc:0.9172|iou:0.8568
36|acc:0.9210|iou:0.8606
37|acc:0.9220|iou:0.8628
38|acc:0.913

In [0]:
run_script(
    "fashion_masks.py",
    "\
    --threads 4\
    --epochs 65\
    --batch_size 256\
    --logname default_CB_shorter_common_most_epochs\
    --learning_rate 0.0015 --learning_rate_final 0.0001\
    --cnn_c M-3-2,CB-256-3-1-same,CB-256-3-1-same,M-3-2,C-512-3-1-same,CB-512-3-1-same,M-3-2\
    --cnn_m CB-256-3-1-same,CB-256-3-1-same,F,R-2048,D\
    --cnn_l F,R-512,D\
    "
)


[INFO]: Found GPU at: /device:GPU:0
<<= <=> =>>
0|acc:0.4036|iou:0.2753
1|acc:0.2396|iou:0.1605
2|acc:0.5170|iou:0.3983
3|acc:0.7700|iou:0.6397
4|acc:0.6224|iou:0.4973
5|acc:0.7860|iou:0.6536
6|acc:0.7254|iou:0.6007
7|acc:0.7262|iou:0.5984
8|acc:0.7652|iou:0.6407
9|acc:0.7958|iou:0.6710
10|acc:0.7230|iou:0.6006
11|acc:0.7878|iou:0.6666
12|acc:0.8256|iou:0.6995
13|acc:0.7902|iou:0.6725
14|acc:0.8308|iou:0.7144
15|acc:0.8074|iou:0.6875
16|acc:0.8132|iou:0.7014
17|acc:0.8304|iou:0.7135
18|acc:0.8252|iou:0.7110
19|acc:0.8316|iou:0.7184
20|acc:0.8304|iou:0.7136
21|acc:0.8472|iou:0.7362
22|acc:0.8374|iou:0.7220
23|acc:0.8312|iou:0.7162
24|acc:0.8504|iou:0.7402
25|acc:0.8528|iou:0.7415
26|acc:0.8344|iou:0.7205
27|acc:0.8492|iou:0.7369
28|acc:0.8528|iou:0.7413
29|acc:0.8514|iou:0.7420
30|acc:0.8518|iou:0.7425
31|acc:0.8534|iou:0.7445
32|acc:0.8582|iou:0.7495
33|acc:0.8518|iou:0.7426
